 ## Описание таблиц

`TRAIN.CSV:`

Это основная таблица, которая связывает студента с вопросом или лекцией, с которыми он взаимодействует, а также дополнительная информация
об этом взаимодействии. Список колонок и краткое их описание:

* row_id: (int64) порядковый номер строки в этой таблице.
* timestamp: (int64) время в миллисекундах между текущим взаимодействием и первым событием связанным с данным ученика.
* user_id: (int32) идентификационный код ученика.
* content_id: (int16) идентификационный код вопроса или лекции.
* content_type_id: (int8) 0 если это вопрос, 1 если лекция.
11
* task_container_id: (int16) идентификационный код группы вопросов, в которой задан текущий вопрос.
* user_answer: (int8) вариант ответа, который выбрал ученик. Всего вариантов может быть 4. В случае если взаимодействие - это лекция, а не вопрос, то данное поле равно -1.
* answered_correctly: (int8) правильно ли ученик ответил на вопрос. В случае, если взаимодействие - это лекция, а не вопрос, то данное поле равно -1. Данное поле нужно будет предсказывать.
* prior_question_elapsed_time: (float32) среднее время в миллисекундах, которое ученику потребовалось, чтобы ответить на все вопросы в предыдущей группе, игнорируя любые лекции между ними. Равен 0 для первой группы вопросов, заданной ученику.
* prior_question_had_explanation: (bool) смотрел ли ученик объяснение и правильный ответ на предыдущий вопрос. Значение равно null, если ученик отвечает на первый вопрос

`QUESTIONS.CSV:`

Таблица описывающая вопрос. Количество вопросов в отличии от количества учеников заранее зафиксировано, также задания повторно предлагаются
различным ученикам. Список колонок:

* question_id: внешний ключ(foreign key) для связывание данной информации с таблицей train.
* bundle_id: код по которому вопросы объединяются в группы.
* correct_answer: правильный ответ на вопрос.
* part: один из семи типов вопросов TOEIC. Ниже будет краткое описание каждого
* tags: один или несколько кодов с тэгами для данного вопроса. Описание тэгов не представлено, но самих кодов достаточно для того, чтобы
сделать кластеризацию.

`LECTIONS.CSV`

Данная таблица аналогична таблице про вопросы, но она про лекции,
перейдем к ее описанию:

* lecture_id: внешний ключ(foreign key) для связывание данной информации с таблицей train.
* type_of: краткое описание основной цели лекции.
* part: один из семи типов, который был в таблице с вопросами.
* tag: один тэг в качестве числа, который представляет лекцию, его значение не дано, также как и в вопросах.


## 1. Таблица Train


In [45]:
import pandas as pd
import numpy as np

In [3]:
# Загрузим данные из файла
# Поскольку объем данных слишком большой для обработки, ограничимся 10000000 строк
# Сразу преобразуем некоторые типы данных
train_0 = pd.read_csv('data/train.csv', sep=',', index_col=0, nrows=10000000,
                       dtype = {'row_id': 'int64', 'timestamp': 'int64', 'user_id': 'int32', 'content_id': 'int16', 
                                'content_type_id': 'int8', 'task_container_id': 'int16', 'user_answer': 'int8', 
                                'answered_correctly': 'int8', 'prior_question_elapsed_time': 'float32', 
                                'prior_question_had_explanation': 'boolean'
                               }
                      )

In [4]:
# Для удобства созданим копию изначального DataFrame
train = train_0.copy()

In [ ]:
# Столбец 'row_id' является индексом и для анализа данных нам не понадобится
train.drop('row_id', axis=1, inplace=True)

In [5]:
train.head()

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
row_id,,,,,,,,,
0,0,115,5692,0,1,3,1,NaN,<NA>
1,56943,115,5716,0,2,2,1,37000.0,False
2,118363,115,128,0,0,0,1,55000.0,False
3,131167,115,7860,0,3,0,1,19000.0,False
4,137965,115,7922,0,4,1,1,11000.0,False


In [6]:
# Проверим информацию по столбцам таблицы
#  Видим, что информация о количестве уникальных значений не отобраилась, видимо, из за большого объема информации
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000000 entries, 0 to 9999999
Data columns (total 9 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   timestamp                       int64  
 1   user_id                         int32  
 2   content_id                      int16  
 3   content_type_id                 int8   
 4   task_container_id               int16  
 5   user_answer                     int8   
 6   answered_correctly              int8   
 7   prior_question_elapsed_time     float32
 8   prior_question_had_explanation  boolean
dtypes: boolean(1), float32(1), int16(2), int32(1), int64(1), int8(3)
memory usage: 314.7 MB


In [8]:
# Выыедем информацию о количесве значений

for i, column in enumerate(train.columns):
    print(f'{i} {column}: {train.shape[0] - train[column].isna().sum()} non-null')
    if train[column].isna().sum() > 0:
        print(f'\t{column}: {train[column].isna().sum()} is NaN')
        
# Столбец 'prior_question_elapsed_time' содержит 1_161_187 пустых значений NaN (для первого набора вопросов или лекций)
# Столбец 'prior_question_had_explanation' имеет 192_960 пустых значений NaN (для первого набора вопросов или лекций)
# Пустые значения NaN можно заменить, например, на среднее значение признаков в данной категории, либо,
# если датасет большой, а пустых значений немного, можно просто ими пренебречь        

0 timestamp: 10000000 non-null
1 user_id: 10000000 non-null
2 content_id: 10000000 non-null
3 content_type_id: 10000000 non-null
4 task_container_id: 10000000 non-null
5 user_answer: 10000000 non-null
6 answered_correctly: 10000000 non-null
7 prior_question_elapsed_time: 9765216 non-null
	prior_question_elapsed_time: 234784 is NaN
8 prior_question_had_explanation: 9960612 non-null
	prior_question_had_explanation: 39388 is NaN


In [10]:
# Описательная статистика таблицы 'train'
# Среднее время ответа около 25 секунд, а максимальное - 5 минут
train.describe()

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time
count,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,9.765216e+06
mean,7.546528e+09,1.079943e+08,5.209509e+03,1.953960e-02,8.776194e+02,1.376462e+00,6.262029e-01,2.547560e+04
std,1.141381e+10,6.311024e+07,3.867783e+03,1.384117e-01,1.334407e+03,1.193603e+00,5.226395e-01,1.995033e+04
min,0.000000e+00,1.150000e+02,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00
25%,5.109600e+08,5.398177e+07,2.059000e+03,0.000000e+00,1.020000e+02,0.000000e+00,0.000000e+00,1.600000e+04
50%,2.612310e+09,1.067638e+08,5.016000e+03,0.000000e+00,3.710000e+02,1.000000e+00,1.000000e+00,2.100000e+04
75%,9.751909e+09,1.630735e+08,7.402000e+03,0.000000e+00,1.055000e+03,3.000000e+00,1.000000e+00,2.966600e+04
max,8.470816e+10,2.167479e+08,3.273600e+04,1.000000e+00,9.999000e+03,3.000000e+00,1.000000e+00,3.000000e+05


In [6]:
# Проверим количесвто уникальных значений для каждой колонки
unique_list = []
for col in train.columns:
    item = (col, train[col].nunique(), train[col].dtype)
    unique_list.append(item)
unique_counts = pd.DataFrame(unique_list,
                             columns=['Column_Name', 'Num_Unique', 'Type']
                            ).sort_values(by='Num_Unique',  ignore_index=True)
display(unique_counts)

,Column_Name,Num_Unique,Type
0,content_type_id,2,int8
1,prior_question_had_explanation,2,boolean
2,answered_correctly,3,int8
3,user_answer,5,int8
4,prior_question_elapsed_time,2583,float32
5,task_container_id,10000,int16
6,content_id,13749,int16
7,user_id,39491,int32
8,timestamp,7534842,int64


In [11]:
display(train['content_type_id'].value_counts(normalize=True))
# 98% активновти - ответы на вопросы
# 2%  активности - лекции

0    0.98046
1    0.01954
Name: content_type_id, dtype: float64

In [13]:
display(train[train['answered_correctly'] != -1]['answered_correctly'].value_counts(normalize=True)) 
display(train[train['answered_correctly'] != -1]['answered_correctly'].mean())

# 66% - правильные ответы 
# 34% - неправильные ответы
# 0.658 - средний балл студента

1    0.658612
0    0.341388
Name: answered_correctly, dtype: float64

0.6586115053703342

In [21]:
display(train[train['content_type_id'] == 0].shape[0] / train['user_id'].nunique())
display(train[train['answered_correctly'] != -1]['answered_correctly'].sum() / train['user_id'].nunique())
display(train[train['content_type_id'] == 1].shape[0] / train['user_id'].nunique())
display(train['prior_question_had_explanation'].sum() / train['user_id'].nunique())
display(train['prior_question_elapsed_time'].mean())

lectures_count = train['content_type_id'].value_counts()[1]
questions_count = train['content_type_id'].value_counts()[0]
display(round(questions_count / lectures_count, 2))


# Cреднее число отвеченных вопросов: 248
# Среднее кол-во правильных ответов: 163
# Среднее число просмотренных лекций: 5
# Среднее число виденных объяснений: 224
# Среднее время, затрачиваемое на вопрос: 25475
# Среднее количество вопросов, приходящихся на одну лекцию - 50.18

248.27439163353677

163.51637081866755

4.947861538071966

224.24235901851054

25475.598

50.18

In [20]:
# Проверим зависимости от времени с начала сессии, среднего времеми ответа студента на вопросы 
#  и влияет ли на результат то, что судент видел ответ на предыдущее задание

pd.DataFrame({'timestamp': train[train['answered_correctly'] != -1]['timestamp'].groupby(train['answered_correctly']).mean(),
              'prior_questions_time': train[train['answered_correctly'] != -1]['prior_question_elapsed_time'].groupby(train['answered_correctly']).mean(),
              'had_explanation': train[train['answered_correctly'] != -1]['prior_question_had_explanation'].groupby(train['answered_correctly']).sum()
             }
            )

# Выводы:
# - чем ближе к началу сессии вопрос, тем менее успешно его проходят студенты 
# - студенты, тратящие меньше времени на решение, чаще отвечают правильно
# - если студент видел объяснение предыдущий задачи, вероятность правильного ответа удваивается

,timestamp,prior_questions_time,had_explanation
answered_correctly,,,
0,7.046183e+09,25704.230469,2880081
1,7.801606e+09,25357.033203,5975474


In [42]:
# Если пользователь думал над вопросом по времени больше среднего значения,
# верность данного ответа на вопрос увеличивалась до 84 %
avg_time = train['prior_question_elapsed_time'].mean()
print('Среднее время ответа на вопрос:', avg_time)
train[(train['prior_question_elapsed_time'] > avg_time) &
         (train['answered_correctly'] == 0) | 
         (train['answered_correctly'] == 1)]['answered_correctly'].value_counts(normalize=True)

Среднее время ответа на вопрос: 25475.598


1    0.844297
0    0.155703
Name: answered_correctly, dtype: float64

In [46]:
# Вопросы, на которые пользователи потратили больше всего времени в секундах (первые пять в порядке убывания)

pd.pivot_table(
    train[train['content_type_id'] == 0],
    values='prior_question_elapsed_time',
    index='content_id',
    aggfunc=np.sum
).sort_values('prior_question_elapsed_time', ascending=False).head()

,prior_question_elapsed_time
content_id,
4120,539459200.0
7216,510579968.0
7217,510579968.0
7218,510579968.0
7219,510579968.0


In [48]:
# Самые популярные лекции среди пользователей платформы по количеству просмотров (первые пять в порядке убывания)
train[train['content_type_id'] == 1]['content_id'].value_counts().head()

3153     3631
21411    2670
29695    2431
10540    2415
15888    2254
Name: content_id, dtype: int64

In [50]:
# Самые популярные вопросы на платформе
train[train['content_type_id'] == 0]['content_id'].value_counts().head()

6116    21103
6173    20060
4120    19826
175     19520
7876    18886
Name: content_id, dtype: int64

In [52]:
# Самые сложные вопросы на платформе (первые пять в порядке убывания)
train[(train['content_type_id'] == 0) & (train['answered_correctly'] == 0)]['content_id'].value_counts().head()

6116    15163
2063    14646
4120    14284
6173    14107
2946    13027
Name: content_id, dtype: int64

In [53]:
# Процент правильных и неправильных ответов на самый сложный вопрос
train[(train['content_type_id'] == 0) & (train['content_id'] == 6116)]['answered_correctly'].value_counts(normalize=True)
# Неправильных ответов 72%

0    0.718523
1    0.281477
Name: answered_correctly, dtype: float64

In [55]:
# Более 82 % ответов на самый лёгкий вопрос являются верными 
train[(train['content_type_id'] == 0) & (train['content_id'] == 7900)]['answered_correctly'].value_counts(normalize=True)

1    0.82566
0    0.17434
Name: answered_correctly, dtype: float64

## 2. questions
Исследуем данные "Вопросы"
questions.csv



In [34]:
questions = pd.read_csv('data/questions.csv', sep=',',
                        dtype = {'question_id': 'int16', 'bundle_id': 'int16', 'correct_answer': 'int8',
                                 'part': 'int8', 'tags': 'object'
                               })
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13523 entries, 0 to 13522
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     13523 non-null  int16 
 1   bundle_id       13523 non-null  int16 
 2   correct_answer  13523 non-null  int8  
 3   part            13523 non-null  int8  
 4   tags            13522 non-null  object
dtypes: int16(2), int8(2), object(1)
memory usage: 185.0+ KB


In [36]:
# выведем количество уникальных значений для каждой колонки
unique_list_q = []
for col in questions.columns:
    item = (col, questions[col].nunique(), questions[col].dtype)
    unique_list_q.append(item)
unique_counts_q = pd.DataFrame(unique_list_q,
                               columns=['Column_Name', 'Num_Unique', 'Type']
                              ).sort_values(by='Num_Unique',  ignore_index=True)
display(unique_counts_q)

,Column_Name,Num_Unique,Type
0,correct_answer,4,int8
1,part,7,int8
2,tags,1519,object
3,bundle_id,9765,int16
4,question_id,13523,int16


In [37]:
questions['question_id'].groupby(questions['bundle_id']).count().max()
questions['part'].value_counts(normalize=True)
questions['tags'].describe()

# Часть вопросов объединены по bundle_id в блоки до 5 вопросов, хотя большинство представлены по одиночке
# Задачи разделены по темам на 7 разделов, больше всего посвящено 5му разделу, 2, 3 и 4

count     13522
unique     1519
top           8
freq        738
Name: tags, dtype: object

In [39]:
# Создадим единую таблицу c ID с данными Train

tmp_df = train.loc[(train.content_type_id == 0), ['content_id', 'answered_correctly']]

In [40]:
# Столбец question_id:

q_list = list(tmp_df['content_id'].unique())
len(q_list)

# Всего 13500 уникальных вопросов. Мы берем этот список из файла Train, чтобы иметь правильный порядок данных

13500

In [56]:
# Cтолбцы :
# q_quant - общее количество ответов для каждого id
# correct_quant - количество правильных ответов для каждого id

q_quant = []
correct_quant = []
for q in q_list:
    tmp = tmp_df[tmp_df['content_id'] == q]['answered_correctly'].count()
    tmp1 = tmp_df[tmp_df['content_id'] == q]['answered_correctly'].sum()
    q_quant.append(tmp)
    correct_quant.append(tmp1)

In [57]:
# Запишем полученные данные в единую таблицу

q_ex = pd.DataFrame({'question_id': q_list,
                        'q_quant': q_quant,
                        'correct_quant': correct_quant
                       }
                      )
q_ex = q_ex.astype({'question_id': 'int16', 'q_quant': 'int32', 'correct_quant': 'int32'})

In [58]:
# Внесем полученные данные в таблицу questions (пересечением),
# вначале удалив ненужные столбцы

questions = questions.drop('correct_answer', axis=1)
questions = pd.merge(questions, q_ex, how='inner')

In [59]:
# Добавим дополнительный столбец:
# Процент правильных ответов

questions['correct_percent'] = questions['correct_quant'] / questions['q_quant']
questions.describe()

,question_id,bundle_id,part,q_quant,correct_quant,correct_percent
count,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000,13500.000000
mean,6760.953778,6760.465556,4.262296,726.266963,478.327778,0.712115
std,3905.579569,3905.539487,1.651777,1142.219033,642.560321,0.169706
min,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,3377.750000,3377.250000,3.000000,173.000000,122.000000,0.608498
50%,6757.500000,6757.000000,5.000000,466.000000,317.000000,0.735702
75%,10147.250000,10145.000000,5.000000,849.000000,585.000000,0.840664
max,13522.000000,13522.000000,7.000000,21103.000000,14866.000000,1.000000


In [60]:
questions

,question_id,bundle_id,part,tags,q_quant,correct_quant,correct_percent
0,0,0,1,51 131 162 38,691,628,0.908828
1,1,1,1,131 36 81,726,649,0.893939
2,2,2,1,131 101 162 92,4501,2531,0.562319
3,3,3,1,131 149 162 29,2276,1776,0.780316
4,4,4,1,131 5 162 38,3153,1980,0.627973
...,...,...,...,...,...,...,...
13495,13518,13518,5,14,86,68,0.790698
13496,13519,13519,5,8,86,50,0.581395
13497,13520,13520,5,73,82,56,0.682927
13498,13521,13521,5,125,87,68,0.781609


In [61]:
# Посмотрим дополнительные условия, от которых может зависеть успешность ответа на вопрос:
# - от номера раздела в тесте TOEIC ('part')

questions.groupby('part').mean()['correct_percent'].sort_values()

# По мере продвижения по разделам, видимо, сложность курса возрастает. 
# 5й раздел содержит максимальное число леций и заданий, однако средняя успеваемость студентов
# для него самая низкая.

/var/folders/m0/bln5llt92_x2w62sl09pg2wc0000gp/T/ipykernel_2419/3082401371.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  questions.groupby('part').mean()['correct_percent'].sort_values()


part
5    0.665941
6    0.699149
7    0.723305
4    0.732642
2    0.748136
3    0.751740
1    0.819349
Name: correct_percent, dtype: float64

In [81]:
lectures.describe()

,lecture_id,tag,part,l_quant
count,402.000000,402.000000,402.000000,402.000000
mean,16956.480100,94.741294,4.201493,802.497512
std,9416.787393,53.133415,1.876936,1077.994893
min,89.000000,0.000000,1.000000,1.000000
25%,9183.500000,52.000000,2.000000,177.250000
50%,17020.000000,95.000000,5.000000,457.500000
75%,24906.250000,139.750000,5.750000,1077.500000
max,32736.000000,187.000000,7.000000,13548.000000


In [82]:
lectures.describe(include=['object'])

,type_of
count,402
unique,3
top,concept
freq,212


In [ ]:
## Исследуем данные "Лекции"

lectures.csv

- lecture_id внешний ключ лекции - соответствует content_id
- part: код темы/раздела 7 тем
- tag: кодировка типа лекции, можно использовать для кластеризации лекций 151 тип
- type_of: краткое описание цели леции: 'concept', 'solving question', 'intention', 'starter'

In [63]:
lectures = pd.read_csv('data/lectures.csv', sep=',',
                      dtype = {'lecture_id': 'int16', 'tag': 'int16', 
                               'part': 'int8', 'type_of': 'object'})
lectures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lecture_id  418 non-null    int16 
 1   tag         418 non-null    int16 
 2   part        418 non-null    int8  
 3   type_of     418 non-null    object
dtypes: int16(2), int8(1), object(1)
memory usage: 5.4+ KB


In [78]:
lectures.describe()

,lecture_id,tag,part,l_quant
count,402.000000,402.000000,402.000000,402.000000
mean,16956.480100,94.741294,4.201493,802.497512
std,9416.787393,53.133415,1.876936,1077.994893
min,89.000000,0.000000,1.000000,1.000000
25%,9183.500000,52.000000,2.000000,177.250000
50%,17020.000000,95.000000,5.000000,457.500000
75%,24906.250000,139.750000,5.750000,1077.500000
max,32736.000000,187.000000,7.000000,13548.000000


In [80]:
lectures.describe(include=['object'])

,type_of
count,402
unique,3
top,concept
freq,212


In [64]:
# Исследуем кол-во уникальных значений для разных колонок

unique_list_lec = []
for col in lectures.columns:
    item = (col, lectures[col].nunique(), lectures[col].dtype)
    unique_list_lec.append(item)
unique_counts_lec = pd.DataFrame(unique_list_lec,
                                 columns=['Column_Name', 'Num_Unique', 'Type']
                                ).sort_values(by='Num_Unique',  ignore_index=True)
display(unique_counts_lec)

# Лекции разделены по темам на 7 разделов, больше всего лекций посвящено 5му разделу, затем 6, 2 и 1
# С учетом данных по вопросам, можно прийти к заключению, что 5 раздел - самый насыщенный по материалу,
# 6 и 1 - более теоретические, а 3 и 4 - более прикладные.
# Леции бывают 4 типов: вступление, целеполагание, концептуальное изложение материала и решение задач.
# Большинство лекций посвящены теории, немного меньше - решению задач. Доля остальных несущественна.
# Есть 151 доп.тип лекций, по которым можно провести кластеризацию

,Column_Name,Num_Unique,Type
0,type_of,4,object
1,part,7,int8
2,tag,151,int16
3,lecture_id,418,int16


In [65]:
lectures['part'].value_counts(normalize=True)
lectures['type_of'].value_counts(normalize=True)
lectures['tag'].value_counts().head()

136    7
27     6
161    6
74     6
134    6
Name: tag, dtype: int64

In [66]:
# Создадим единую таблицу по их ID с данными Train.

l_list = list(train[train['content_type_id'] == 1]['content_id'].unique())


402

In [74]:
# Посчитаем количество просмотров лекций для каждого id

l_quant = []
for l in l_list:
    tmp_l = train[(train['content_id'] == l)]['content_id'].count()
    l_quant.append(tmp_l)

In [75]:
# Запишем полученные данные в единую таблицу

l_ex = pd.DataFrame({'lecture_id': l_list,
                        'l_quant': l_quant
                       }
                      )
l_ex = l_ex.astype({'lecture_id': 'int16', 'l_quant': 'int32'})

In [76]:
# Внесем полученные данные в таблицу lectures (пересечением)

lectures = pd.merge(lectures, l_ex, how='inner')
display(lectures)

,lecture_id,tag,part,type_of,l_quant
0,89,159,5,concept,2437
1,100,70,1,concept,1794
2,185,45,6,concept,4701
3,192,79,5,solving question,1111
4,317,156,5,solving question,1067
...,...,...,...,...,...
397,32491,179,5,solving question,163
398,32535,8,5,solving question,348
399,32570,113,3,solving question,285
400,32625,142,2,concept,774


In [70]:
# Общий рейтинг популярности для лекций

lectures.sort_values(by=['l_quant'], ascending=False).head()

,lecture_id,tag,part,type_of,l_quant
121,10688,39,7,concept,13548
5,335,114,2,concept,5952
44,3852,161,4,solving question,5754
2,185,45,6,concept,4701
61,5694,140,2,concept,4468


In [73]:
# Рейтинг популярности для лекций, в зависимости от тега лекции
lectures.groupby('tag').sum()['l_quant'].sort_values(ascending=False).head()

/var/folders/m0/bln5llt92_x2w62sl09pg2wc0000gp/T/ipykernel_2419/1620134833.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  lectures.groupby('tag').sum()['l_quant'].sort_values(ascending=False).head()


tag
39     13548
161     9147
27      7995
62      7900
114     6803
Name: l_quant, dtype: int32

## Выводы:

- Среднее время ответа около 25 секунд
- Максимальное время ответа - 5 минут
- 98% активновти студентов приходится на ответы на вопросы 
- Только 2% татится на просмотр лекций
- 66% составляют правильные ответы, 34% - неправильные.
- 0.658 - средний балл студента
- Cреднее число отвеченных вопросов: 248
- Среднее кол-во правильных ответов: 163
- Среднее число просмотренных лекций: 5
- Среднее число виденных объяснений: 224
- Среднее время, затрачиваемое на вопрос: 25475
- Среднее количество вопросов, приходящихся на одну лекцию - 50.18
- чем ближе к началу сессии вопрос, тем менее успешно его проходят студенты орме)
- Студенты, отвечающие правильно, тратят меньше времени на решение
- Если студент видел объяснение предыдущий задачи, вероятность правильного ответа удваивается
- Если пользователь думал над вопросом по времени больше среднего значения
- Больше всего времени пользователи потратили на вопросы 4120, 7216, 7217
- Самые популярные лекции 3151, 21441, 29695
- Самые популярные вопросы на платформе 6116, 6173, 4120
- Самые сложные вопросы на латформе 6116, 2063, 4120
- На самый сложный вопрос учащиеся дали 72% неправильных ответов и 28% правильных
- Около 82% ответов на самый легкий вопрос являются верными и 18 неверными
- Часть вопросов объединены по bundle_id в блоки до 5 вопросов, хотя большинство представлены по одиночке
- Лекции бывают 4 типов: вступление, целеполагание, концептуальное изложение материала и решение задач.
- Задачи и лекции разделены по темам на 7 разделов
- Большинство лекций посвящены теории, немного меньше - решению задач.
- Наибольшее число просмотров имеет лекция из 7го раздела.
- Самая высокая средневзвешенная популярность у лекций 2 раздела.
- 5 раздел - самый насыщенный по материалу, 6 и 1 - более теоретические, а 3 и 4 - более прикладные.
- По мере продвижения по разделам, видимо, сложность курса возрастает.
- 5й раздел содержит максимальное число лекций и заданий, однако средняя успеваемость студентов для него самая низкая.
- Успеваемость по 4му и 6му разделу, вероятно, можно улучшить, добавив в них качественные лекции, а по 7му разделу - добавив практические задания.